
{'pstring': '014/14/2/3', 'transimpedance': 727.2621911621392, 'Vpp': 0.004927079615255271, 'Ref_Vpp': 0.0, 'output_impedance': 291.38209297866723, 'current_bias': 0.8785819282317335}


'pstring' Pathstrings: '014/14/1/1' record name
 
'transimpedance': 738.6014767733689, slope (= dV/dI) 

'Vpp': 0.004890991306172068 high of function




In [14]:
import pickle as pkl

def getDict(file):
    with open(file,'rb') as f:
        SQUID_dict = pkl.load(f)
        print(type(SQUID_dict.keys()))
    return SQUID_dict

file_20230911_222403 = '20230911_222403_SQUIDParams.pkl'
file_20230919_212918 = '20230919_212918_SQUIDParams.pkl'
file_20230919_221550 = '20230919_221550_SQUIDParams.pkl'
file_20230919_234736 = '20230919_234736_SQUIDParams.pkl'
file_20230920_014329 = '20230920_014329_SQUIDParams.pkl'
file_20230920_031303 = '20230920_031303_SQUIDParams.pkl'

dict_20230911_222403 = getDict(file_20230911_222403)
dict_20230919_212918 = getDict(file_20230919_212918)
dict_20230919_221550 = getDict(file_20230919_221550)
dict_20230919_234736 = getDict(file_20230919_234736)
dict_20230920_014329 = getDict(file_20230920_014329)
dict_20230920_031303 = getDict(file_20230920_031303)



<class 'dict_keys'>
<class 'dict_keys'>
<class 'dict_keys'>
<class 'dict_keys'>
<class 'dict_keys'>
<class 'dict_keys'>


In [17]:
# get all same exictance keys in all dictionaries
same = list(dict_20230911_222403.keys()) 
print(type(same))

<class 'list'>
